## Curso: Proyecto Integrador
### Integrantes:
#### Andrea Carolina Garza Nacif A01280427
#### Jhamyr Arnulfo Alcalde Oballe A01795409
#### Víctor Arista Beltrán A01012223

In [1]:
#Cargar y preparar los datos
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from IPython.display import Markdown

csv_file = '/Users/andreagarzanacif/Desktop/sparkvista_dataset.csv'
df = pd.read_csv(csv_file)

In [2]:
#Pre-procesamiento
for col in ["Year Founded", "IPO Date"]:
    df[col] = df[col].astype(str).str.extract(r'(\d{4})')[0].astype("Int64")
    df[col] = pd.to_datetime(df[col], format='%Y', errors='coerce')
df["Company_Age"] = 2025 - df["Year Founded"].dt.year

#Target
fund_q75 = df["Total Funding Amount (USD)"].quantile(0.75)
df["HighPotential"] = (
    (df["Total Funding Amount (USD)"] >= fund_q75) |
    (df["Funding Status"].str.contains("IPO|Public|Acquired", na=False))
).astype(int)

y = df["HighPotential"].values
X = df.drop(columns=["HighPotential", "Total Funding Amount (USD)"])

In [3]:
#Columnas numéricas y categóricas
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

numeric_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  StandardScaler())
])
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
categorical_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])
prep = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

# Split
X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

In [4]:
#Entrenar y comparar modelos

modelos = {
    "Logistic Regression": LogisticRegression(max_iter=1000, C=1.0),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Naive Bayes": GaussianNB(),
    "SVM (Linear)": SVC(kernel="linear", probability=True, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
}

resultados = []
for nombre, clf in modelos.items():
    pipe = Pipeline([("prep", prep), ("clf", clf)])
    pipe.fit(X_tr, y_tr)
    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        proba = pipe.predict_proba(X_te)[:, 1]
    else:
        proba = pipe.decision_function(X_te)
    auc = roc_auc_score(y_te, proba)
    resultados.append({"Modelo": nombre, "AUC-ROC": auc, "Pipeline": pipe})

df_resultados = pd.DataFrame(resultados).sort_values("AUC-ROC", ascending=False)
display(df_resultados[["Modelo", "AUC-ROC"]])

/Users/andreagarzanacif/env-pyspark/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [23:13:34] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,Modelo,AUC-ROC
0,Logistic Regression,0.845761
4,SVM (Linear),0.838996
2,Random Forest,0.834041
6,XGBoost,0.795422
5,KNN,0.784175
1,Decision Tree,0.689791
3,Naive Bayes,0.620576


In [5]:
#Ajuste de los dos mejores modelos
top2 = df_resultados.head(2)
param_grids = {
    "Random Forest": {
        "clf__n_estimators": [100, 200],
        "clf__max_depth": [None, 5, 10]
    },
    "Logistic Regression": {
        "clf__C": [0.1, 1.0, 10]
    },
    "XGBoost": {
        "clf__n_estimators": [50, 100],
        "clf__max_depth": [3, 6],
        "clf__learning_rate": [0.01, 0.1, 0.3]
    },
    "SVM (Linear)": {
        "clf__C": [0.1, 1.0, 10]
    },
    "Decision Tree": {
        "clf__max_depth": [None, 5, 10],
        "clf__min_samples_split": [2, 5, 10]
    },
    "KNN": {
        "clf__n_neighbors": [3, 5, 7]
    },
    "Naive Bayes": {}
}

tuned_models = []
for ix, row in top2.iterrows():
    nombre = row["Modelo"]
    print(f"\nAjustando hiperparámetros para: {nombre}")
    grid = GridSearchCV(
        Pipeline([("prep", prep), ("clf", modelos[nombre])]),
        param_grids[nombre],
        cv=5,
        scoring="roc_auc",
        n_jobs=-1,
        verbose=2
    )
    grid.fit(X_tr, y_tr)
    auc_te = roc_auc_score(y_te, grid.best_estimator_.predict_proba(X_te)[:, 1])
    tuned_models.append({
        "Modelo": nombre + " (tuned)",
        "AUC-ROC": auc_te,
        "Pipeline": grid.best_estimator_,
        "Params": grid.best_params_
    })
    print(f"Mejor AUC-ROC (val): {grid.best_score_:.3f} | AUC-ROC (test): {auc_te:.3f}")
    print(f"Mejores hiperparámetros: {grid.best_params_}")


Ajustando hiperparámetros para: Logistic Regression
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Mejor AUC-ROC (val): 0.853 | AUC-ROC (test): 0.843
Mejores hiperparámetros: {'clf__C': 10}

Ajustando hiperparámetros para: SVM (Linear)
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Mejor AUC-ROC (val): 0.855 | AUC-ROC (test): 0.839
Mejores hiperparámetros: {'clf__C': 1.0}


In [6]:
#Tabla de resultados
df_resultados = pd.concat([df_resultados, pd.DataFrame(tuned_models)], ignore_index=True)
df_resultados = df_resultados.sort_values("AUC-ROC", ascending=False)
display(df_resultados[["Modelo", "AUC-ROC"]])

,Modelo,AUC-ROC
0,Logistic Regression,0.845761
7,Logistic Regression (tuned),0.842772
1,SVM (Linear),0.838996
8,SVM (Linear) (tuned),0.838996
2,Random Forest,0.834041
3,XGBoost,0.795422
4,KNN,0.784175
5,Decision Tree,0.689791
6,Naive Bayes,0.620576


In [7]:
#Conclusión
modelo_final = df_resultados.iloc[0]
print(f"\n>>> Modelo individual final seleccionado: {modelo_final['Modelo']} (AUC-ROC = {modelo_final['AUC-ROC']:.3f})")

conclusion = f"""
## Conclusiones del Experimento

Se entrenaron y compararon múltiples modelos de clasificación binaria para anticipar startups con alto potencial. 
Los resultados muestran lo siguiente:

- El modelo de mejor desempeño fue **{modelo_final['Modelo']}**, con un AUC-ROC en test de **{modelo_final['AUC-ROC']:.3f}**.
- La comparación sistemática demostró que la selección del modelo y el ajuste de hiperparámetros pueden incrementar significativamente el rendimiento frente a modelos sin ajuste.
- Este análisis refuerza la importancia de experimentar con diversos algoritmos y configuraciones para encontrar la solución óptima para el problema específico.

> *El modelo seleccionado se recomienda como base para despliegue o para posteriores ajustes y validaciones con nuevos datos.*
"""
display(Markdown(conclusion))


>>> Modelo individual final seleccionado: Logistic Regression (AUC-ROC = 0.846)



## Conclusiones del Experimento

Se entrenaron y compararon múltiples modelos de clasificación binaria para anticipar startups con alto potencial. 
Los resultados muestran lo siguiente:

- El modelo de mejor desempeño fue **Logistic Regression**, con un AUC-ROC en test de **0.846**.
- La comparación sistemática demostró que la selección del modelo y el ajuste de hiperparámetros pueden incrementar significativamente el rendimiento frente a modelos sin ajuste.
- Este análisis refuerza la importancia de experimentar con diversos algoritmos y configuraciones para encontrar la solución óptima para el problema específico.

> *El modelo seleccionado se recomienda como base para despliegue o para posteriores ajustes y validaciones con nuevos datos.*
